To load and plot a dataset of neural activity across population, in a PopAnal class object.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:

# this is the path to the dataset
path = '/gorilla1/analyses/recordings/main/RSA/Diego-230615/agg_True-subtr_None-dist_euclidian_unbiased/SP_shape_loc/DFallpa.pkl'


##### Load data

In [3]:
DFallpa = pd.read_pickle(path)


# Code example for benchmarking: decoding shapes

This step takes in a representation of neural data and outputs a scalar score for how well you can decode "shape" from that data

Here, this example is using the raw data (dimensionality = number of channels). The goal is to use methods to reduce the dimensionality of this data, each time running through this decoding benchmark, to compare the different methods

##### First, pull out a specific PA. (just an example)

NOTE: tjhis is just for demonstration. Eventually you will want to loop thru all PA, scoring them all

In [6]:
DFallpa

,which_level,event,bregion,twind,pa
0,trial,03_samp,M1_m,"(-0.5, -0.3)",<neuralmonkey.classes.population.PopAnal objec...
1,trial,03_samp,M1_l,"(-0.5, -0.3)",<neuralmonkey.classes.population.PopAnal objec...
2,trial,03_samp,PMv_l,"(-0.5, -0.3)",<neuralmonkey.classes.population.PopAnal objec...
3,trial,03_samp,PMv_m,"(-0.5, -0.3)",<neuralmonkey.classes.population.PopAnal objec...
4,trial,03_samp,PMd_p,"(-0.5, -0.3)",<neuralmonkey.classes.population.PopAnal objec...
...,...,...,...,...,...
145,trial,06_on_strokeidx_0,FP_a,"(0.3, 0.5)",<neuralmonkey.classes.population.PopAnal objec...
146,trial,06_on_strokeidx_0,SMA_p,"(0.3, 0.5)",<neuralmonkey.classes.population.PopAnal objec...
147,trial,06_on_strokeidx_0,SMA_a,"(0.3, 0.5)",<neuralmonkey.classes.population.PopAnal objec...
148,trial,06_on_strokeidx_0,preSMA_p,"(0.3, 0.5)",<neuralmonkey.classes.population.PopAnal objec...


In [5]:
def extract_single_pa(DFallpa, bregion, twind):
    which_level = "trial"
    event = "03_samp"

    a = DFallpa["which_level"]==which_level
    b = DFallpa["event"]==event
    c = DFallpa["bregion"]==bregion
    d = DFallpa["twind"]==twind
    
    tmp = DFallpa[a & b & c & d]
    assert len(tmp)==1
    pa = tmp["pa"].values[0]
    
    return pa
    

In [12]:
### PARAMS
bregion = "PMv_m"
twind = (0.3, 0.5)

### RUN
pa = extract_single_pa(DFallpa, bregion, twind)

nchans = pa.X.shape[0]
ntrials = pa.X.shape[1]
ntimes = pa.X.shape[2]

print("Shape of this dataset (chans, trials, times):", nchans, ntrials, ntimes)
print("Data is stored in pa.X: ", pa.X.shape)

Shape of this dataset (chans, trials, times): 21 422 20
Data is stored in pa.X:  (21, 422, 20)


##### Finally, extract the data that goes into the decoder

Extract activity wthin a specific time bin. Eventualy, you would want to test all time bins.

In [14]:
tbin = 0
X = pa.X[:, :, tbin].T # (ntrials, nchans)
print(X.shape)

(422, 21)


The variable you are trying to decode/predict is the shape on each trial.

In [15]:
var = "seqc_0_shape"
shapes = pa.Xlabels["trials"][var].tolist()

print(shapes[:5])
print(len(shapes))

['Lcentered-4-1-0', 'L|V-2-3-0', 'Lcentered-4-1-0', 'L|V-2-3-0', 'circle-6-1-0']
422


##### Train and test decoder

This is using a helper function I wrote (_model_fit) but you should go into the code to know how it works

In [18]:
from neuralmonkey.population.classify import _model_fit

model_params_optimal = {"C":0.01} # optimized regularization params
pipe, score = _model_fit(X, shapes, model_params=model_params_optimal)

print("Model score (prediction accuracy on held-out test data):", score)

Model score (prediction accuracy on held-out test data): 0.6421800947867299


In [20]:
print("Expected score if guessing = ", 1/len(set(shapes)))

Expected score if guessing =  0.16666666666666666


# NEXT STEPS

1. Try different methods for dim reduction of X. In all cases, if X is shape (nchans, ntrials, ntimes), dim reduction should return Xreduced of shape (ndim, ntrials, ntimes) where ndim < nchans
2. For the benchmark decoder, you want to score looping through each bregion, twind, and tbin. i.e, see below

# [FILL THIS IN] ENTIRE PIPELINE

In [ ]:
list_dim_reduction_methods = ["pca", "CEBRA", ...] # FILL THIS IN

In [ ]:
list_br = DFallpa["bregion"].unique().tolist()
list_tw = DFallpa["twind"].unique().tolist()

res = []

for br in list_br:
    for tw in list_tw:
        # 1. Extract the specific pa for this (br, tw)
        pa = extract_single_pa(DFallpa, br, tw)
        
        # 2. Extract X from pa
        X = pa.X # (nchans, ntrials, ntimes)
        # shapes = pa.Xlabels["trials"][var].tolist()
        list_tbin = range(X.shape[2])
        
        for method in list_dim_reduction_methods:
            # 2. Apply this method to pa
            # Xreduced = dim_reduce(X, method) # FILL THIS IN
            
            for tbin in list_tbin:
                # 3a. Extract data for this time bin
                # Xreduced_this_time_bin = Xreduced[:, :, tbin].T # (ntrials, nchans)
    
                # 3. Run decoder
                # score = shape_decoder(Xreduced_this_time_bin, shapes) # FILL THIS IN
                
                # 3. Collect data
                res.append({
                    "method":method,
                    "bregion":br,
                    "twind":tw,
                    "tbin":tbin,
                    "score":score
                })
                

In [ ]:
##### Plot the results, comparing score across methods!!
# FILL THIS IN